In [1]:
import numpy as np
from copy import copy
import random
import time
import math
import sympy as sy
import itertools
from fractions import Fraction
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
url='https://www.janestreet.com/puzzles/current-puzzle/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([[i,j] for i,j in enumerate(y)])
display(Markdown("### "+y[8]+"\n\n"+str("\n".join(y[9:75]))))

### Hooks 8

The grid above can be partitioned into 9 L-shaped “hooks”. The largest
is 9-by-9 (contains 17 squares), the next largest is 8-by-8 (contains
15 squares), and so on. The smallest hook is just a single
square. Find where the hooks are located, and place nine 9’s in one of
the hooks, eight 8’s in another, seven 7’s in another, and so on.
The filled squares must form a connected region. (Squares are
“connected” if they are orthogonally adjacent.) Furthermore, every
2-by-2 region must contain at least one unfilled square.
The sum of the values in each of the connected shaded regions must be
the same.
The answer to this puzzle is the product of the areas of the connected
groups of empty squares in the completed grid.
Submit Your Answer
*
Required fields
Name/Username
Email
*
Answer
*
Questions/Comments
I would like to have my name added to the leaderboard.
Please note that only initial submissions will
                        qualify.
Thank you for submitting your answer to the current puzzle.
Please check back before the end of the month to to see if you've correctly solved this month's puzzle.
×
There was an error processing your submission. We apologize for
                    the inconvenience. Please try email
puzzles@janestreet.com
if you continue to experience any issues.
What We Do
Our Philosophy
ETFs
Institutional Services
Electronic Platforms
ETF Trading Survey
Our Reports
Technology
Tech Talks
Tech Blog
Tech Podcast
Code Library
Culture
Benefits
Jane Street U
Puzzles
Join Jane Street
Internships
Experienced Candidates
Interviewing
On-Campus Events
Apply
Contact Us
New York
London
Hong Kong
Amsterdam
Investor Relations
Media Inquiries
Disclosures & Policies
© Copyright 2015-2022 Jane Street Group, LLC. All rights reserved. Services are provided in the U.S. by Jane Street Capital, LLC and Jane Street Execution Services, LLC, each of which is a SEC-registered broker dealer and member of FINRA (
www.finra.org
). Regulated activities are undertaken in Europe by Jane Street Financial Limited, an investment firm authorized and regulated by the U.K. Financial Conduct Authority, and Jane Street Netherlands B.V., an investment firm authorized and regulated by the Netherlands Authority for the Financial Markets (
Autoriteit Financiële Markten
), and in Hong Kong by Jane Street Hong Kong Limited, a regulated entity under the Hong Kong Securities and Futures Commission (CE No. BAL548). Each of these entities is a wholly owned subsidiary of Jane Street Group, LLC. This material is provided for informational purposes only and does not constitute an offer or solicitation for the purchase or sale of any security or other financial instrument. | Jane Street and the concentric circle mark are registered trademarks of Jane Street.
Ad and Cookie Policy
Privacy Policy
Jane Street is an Equal Opportunity Employer

In [3]:
# build out the probability tree
def tree(p,stopper):
    game = [(np.ones(p),Fraction(1,1),1,1)]  # live players,probability, number of shots, next player
    result = []
    
    while len(game) >0:
        (players,prob,num,x) = game.pop()
        if np.sum(players) ==1:
            result.append([np.argmax(players)+1,prob])
        elif num >stopper:
            result.append([0,prob])
        else:
            num +=1
            pass_prob = prob * Fraction(1,num)
            fail_prob = prob * Fraction(num-1,num)
            fail_players = copy(players)
            fail_players[x] = 0
            nextp = (x +1) % p
            while players[nextp] ==0:
                nextp = (nextp +1) % p
            game.append([players,pass_prob,num,nextp])
            game.append([fail_players,fail_prob,num,nextp])
    print(len(result),"total paths")
    return [(n,float(np.sum([j for i,j in result if i==n])))for n in range(p+1)]


In [4]:
output = tree(4,18)
for i in range(5):
    print("Player {} has a {:.10f} probability".format(output[i][0],output[i][1]))

988 total paths
Player 0 has a 0.0000000000 probability
Player 1 has a 0.3711532348 probability
Player 2 has a 0.2421885526 probability
Player 3 has a 0.2032205617 probability
Player 4 has a 0.1834376509 probability


In [5]:
url='https://www.janestreet.com/puzzles/robot-archery-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([[i,j] for i,j in enumerate(y)])
display(Markdown("### "+y[7]+"\n\n"+str("\n".join(y[10:219]))))

### Robot Archery

[Above is a Mathematica-derived solution to the systems of equations
described in the 4-player tournament described below]
Without loss of generality we can assume an arrow’s distance to the center of the target is U[0,1] distributed. Let
P
j,k
(
x
) denote the probability that Player
j
out of
k
in the
current
line will eventually win the game, given that the current best dart is at distance
x
.
In a 2-player archery tournament, we would have
P
1,2
(
x
) +
P
2,2
(
x
) = 1,
P
1,2
(0) = 0, and
P
1,2
(
x
) = ∫
0
x
P
2,2
(
u
) d
u
. Taking derivatives of both sides in that last equation and substituing gives
P
‘
1,2
(
x
) = −
P
1,2
(x). Solving gives
P
1,2
(
x
) = 1−e
−
x
.
In the 3-player version of the game, we get the following system of equations:
P
1,3
(
x
) = ∫
0
x
P
3,3
(
u
) d
u
P
2,3
(
x
) = (1 -
x
)
P
1,2
(
x
) + ∫
0
x
P
1,3
(
u
) d
u
P
1,3
(
x
) +
P
2,3
(
x
) +
P
3,3
(
x
) = 1
P
1,3
(0) =
P
2,3
(0) = 0.
And for the 4-player version we have the following:
P
1,4
(
x
) = ∫
0
x
P
4,4
(
u
) d
u
P
2,4
(
x
) = (1 -
x
)
P
1,3
(
x
) + ∫
0
x
P
1,4
(
u
) d
u
P
3,4
(
x
) = (1 -
x
)
P
2,3
(
x
) + ∫
0
x
P
2,4
(
u
) d
u
P
1,4
(
x
) +
P
2,4
(
x
) +
P
3,4
(
x
) +
P
4,4
(
x
) = 1
P
1,4
(0) =
P
2,4
(0) =
P
3,4
(0) = 0.
In particular we have
P
4,4
(1) = (-5/4)*(cos(1) + sin(1)) + (1/2)*
e
-1
+ (
e
-1/2
)*(cos(sqrt(3)/2) + 5/sqrt(3)*sin(sqrt(3)/2)), or approximately 0.18343765086.
Congrats to this month’s solvers!

In [6]:
#exact solution. 4 player game doesn't simplify to the 3 player game on hit so gets tricky. 2 player game is tractable.
import sympy as sy
n = sy.symbols("n")

print("For the 2 player game the probability of second player winning is ")

form = sy.Sum((1-1/(2*n+1))*1/(sy.factorial(2*n)),(n,1,sy.oo))
print(form)
print(form.doit())


For the 2 player game the probability of second player winning is 
Sum((1 - 1/(2*n + 1))/factorial(2*n), (n, 1, oo))
-sinh(1) + cosh(1)


In [7]:
# simulate to check 
def game(players=4):
    live = np.ones(players)
    distance = np.sum(np.random.rand(2)**2)**0.5   
    player = 1
    while sum(live) > 1:
        if live[player] ==1:
            shot = np.sum(np.random.rand(2)**2)**0.5   
            if shot > distance:
                live[player] = 0
            else:
                distance = shot
        player = (player + 1 ) % players
        while live[player] ==0:
            player = (player + 1 ) % players
    return (live)    

In [8]:
start = time.time()
N= 10**5
for i in range(3):
    print(np.sum([game(4) for i in range(N)],axis=0)/N)
print("Took",time.time()-start)

[0.37166 0.24092 0.20425 0.18317]
[0.37228 0.24309 0.20274 0.18189]
[0.37286 0.24315 0.2006  0.18339]
Took 17.124910831451416
